<a href="https://colab.research.google.com/github/faizanahemad/eva/blob/master/assignment-14/DN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Resources**

- XLA

    - [Google Notebook example](https://colab.research.google.com/github/tensorflow/tensorflow/blob/master/tensorflow/compiler/xla/g3doc/tutorials/xla_compile.ipynb)

    - [Blog](https://medium.com/@xianbao.qian/use-xla-with-keras-3ca5d0309c26)
    - [Example Enabling XLA](https://github.com/tensorflow/models/blob/7212436440eaa11293ca84befcc5d8327109ea76/official/utils/misc/keras_utils.py#L158)
    
- Mixed Precision

- Augmentation Libs
    - [imgaug](https://github.com/aleju/imgaug)
    - [albumentations](https://github.com/albu/albumentations)
    - [Automold](https://github.com/UjjwalSaxena/Automold--Road-Augmentation-Library)
    - [Tensorflow Examples](https://www.wouterbulten.nl/blog/tech/data-augmentation-using-tensorflow-data-dataset/)
    - [PIL ImageOps Examples](https://hhsprings.bitbucket.io/docs/programming/examples/python/PIL/ImageOps.html)
    
- Tensorflow References
    - [Using Numpy functions](https://www.tensorflow.org/api_docs/python/tf/numpy_function)
    - [Using Python functions](https://www.tensorflow.org/api_docs/python/tf/py_function)
    - [Tensorflow Data](https://www.tensorflow.org/datasets/catalog/overview)
    - [Dataset to TFRecord](https://github.com/tensorflow/tensorflow/issues/16926)
    - [TFRecord](https://www.tensorflow.org/tutorials/load_data/tf_records#tfrecords_format_details)
    - [TFRecord Load](https://www.tensorflow.org/tutorials/load_data/images)

# Imports

In [1]:
import os
import gc

jobs = int(os.cpu_count()/2)


In [2]:
import tensorflow as tf
# tf.enable_eager_execution()
# tf.keras.backend.set_epsilon(1e-4)
# tf.keras.backend.set_floatx('float16')

# tf.random.set_random_seed(1234)


/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ec2-user/a

In [3]:
import numpy as np
import skimage
print(np.__version__)
print(skimage.__version__)


import time, math
from tqdm import tqdm_notebook as tqdm

import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.contrib.eager as tfe
import gc
print(tf.__version__)
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
% matplotlib inline
%config InlineBackend.figure_format='retina'

import copy
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from tensorflow.core.protobuf import rewriter_config_pb2
import os
from importlib import reload
import lib
reload(lib)
from lib import *
from data_science_utils import misc
from data_science_utils.vision.keras.regularizers import get_cutout_eraser

import pandas as pd


import imgaug.augmenters as iaa
from matplotlib import cm
from albumentations import (
    HorizontalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine,RandomFog,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, ChannelDropout, ChannelShuffle,RandomContrast
)

1.17.0
0.15.0


W0824 13:35:37.386745 140608712148800 deprecation_wrapper.py:119] From /home/ec2-user/SageMaker/eva/assignment-14/lib.py:207: The name tf.keras.backend.set_session is deprecated. Please use tf.compat.v1.keras.backend.set_session instead.

W0824 13:35:37.387819 140608712148800 deprecation_wrapper.py:119] From /home/ec2-user/SageMaker/eva/assignment-14/lib.py:209: The name tf.keras.backend.get_session is deprecated. Please use tf.compat.v1.keras.backend.get_session instead.



1.14.0


Using TensorFlow backend.


In [4]:
logger.info('This is a warning')


I0824 13:35:37.505244 140608712148800 <ipython-input-4-bf20c9c65a9e>:1] This is a warning


# GridSearch

In [5]:
def read_and_append_to_results(filename, result_object=None, read=False):
    from pathlib import Path
    import ast
    my_file = Path(filename)
    if my_file.is_file():
        results = misc.load_list_per_line(filename)
    else:
        results = []
    
    results = list(filter(lambda x:len(x)>2,results))
    
    if result_object is not None:
        results.append(result_object)
        misc.save_list_per_line(results, filename)
        
    if read:
        def lit_eval(r):
            try:
                return ast.literal_eval(r)
            except:
                return None
        return list(filter(lambda x: x is not None,map(lit_eval,results)))
    else:
        return

In [12]:
def run_model(model,BATCH_SIZE,EPOCHS,train_dataset,train_dataset_one_epoch,x_test, y_test,len_train,log_test_acc=False):
    len_test = len(x_test)
    test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE).prefetch(len_test).cache()
    train_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)
    logger.debug("Starting Training")

    model.fit(train_dataset, epochs=EPOCHS,steps_per_epoch=20)
    
    test_res = model.evaluate(test_dataset)
    train_res = model.evaluate(train_dataset_one_epoch)

    print(train_res,test_res)
    t = time.time()
              
    time_spent = time.time() - t
    logger.debug(msg("Train acc = ",train_accs[-1],"Test acc =",test_accs[-1],"Time Taken = ",time_spent))
    return train_accs[-1],test_accs[-1],time_spent
    
    


In [13]:
class LearningRateScheduler(tf.keras.callbacks.Callback):
    def __init__(self,lr_function,momentum_function):
        super(LearningRateScheduler, self).__init__()
        self.lr_function = lr_function
        self.momentum_function = momentum_function

    def on_train_batch_begin(self, batch, logs=None):
        scheduled_lr = self.lr_function(batch)
        scheduled_momentum = self.momentum_function(batch)
        tf.keras.backend.set_value(self.model.optimizer.lr, scheduled_lr)
        tf.keras.backend.set_value(self.model.optimizer.momentum, scheduled_momentum)
        


In [14]:

def model_builder(model_config,BATCH_SIZE, len_train):
    
    batches_per_epoch = len_train//BATCH_SIZE + 1
    model = FNet(**model_config["model"])
    
    enable_olr = model_config["optimizer"]["enable_olr"]
    enable_momentum = enable_olr
    max_lr = model_config["optimizer"]["max_lr"]
    max_momentum = model_config["optimizer"]["max_momentum"]
    global_step = tf.train.get_or_create_global_step()
    if enable_olr:
        mid_epoch = model_config["optimizer"]["mid_epoch"]
        start_lr = model_config["optimizer"]["start_lr"]
        
        end_lr = model_config["optimizer"]["end_lr"]
        pre_end_lr = model_config["optimizer"]["pre_end_lr"]
        pre_end_epoch = model_config["optimizer"]["pre_end_epoch"]
        
        enable_momentum = model_config["optimizer"]["enable_momentum"]
        if enable_momentum:
            min_momentum = model_config["optimizer"]["min_momentum"]
            momentum_schedule =  lambda t: np.interp([t], [0, mid_epoch, pre_end_epoch, EPOCHS], [max_momentum, min_momentum, max_momentum, max_momentum])[0] 
            momentum_func = lambda batch: momentum_schedule(batch/batches_per_epoch)
    
        lr_schedule = lambda t: np.interp([t], [0, mid_epoch, pre_end_epoch, EPOCHS], [start_lr, max_lr, pre_end_lr, end_lr])[0] # LR = 0.75
        lr_func = lambda batch: lr_schedule(batch/batches_per_epoch)
    
    lrs = LearningRateScheduler(lr_function=lr_func,momentum_function=momentum_func)

    opt = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
    model.compile(optimizer=opt,loss='sparse_categorical_crossentropy',metrics=['sparse_categorical_accuracy'],callbacks=[lrs])
    logger.debug("Model Built")
    return model,opt,global_step


In [15]:
def build_augmenters(da_config,train_mean):
    cutout_config = da_config["cutout_config"]
    cifar10_augs_config = da_config["cifar10_augs_config"]
    hue_config = da_config["hue_config"]
    cutout_mapper = get_numpy_wrapper(get_cutout_eraser(**cutout_config),"cutout")
    cifar10_mapper = get_numpy_wrapper(CIFAR10Policy(**cifar10_augs_config, fillcolor=tuple(train_mean.astype(int)), log=False),"AutoAug")
    hue_mapper = get_hue_aug(hue_config["max_delta"])
    caster = lambda x,y: (tf.cast(x,tf.float32),tf.cast(y,tf.int64))
#     [hflip_mapper,cutout_mapper,cifar10_mapper,hue_mapper,caster]
    full_wrapper = get_multimapper([])
    logger.debug("Augmentation Functions Built")
    return full_wrapper


def process_full_augmentation_all_epochs(full_wrapper, EPOCHS,BATCH_SIZE, normalize, x_train, y_train):
    train_data = {}
    len_train = len(x_train)
    normalize = get_numpy_wrapper(normalize, Tout=tf.float32)
    caster = lambda x,y: (tf.cast(x,tf.float32),tf.cast(y,tf.int64))
    normalize = get_multimapper([])
    t = time.time()
    train_set_one_epoch = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(full_wrapper, num_parallel_calls=tf.data.experimental.AUTOTUNE).map(normalize, num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(BATCH_SIZE).prefetch(len_train)
    train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(full_wrapper, num_parallel_calls=tf.data.experimental.AUTOTUNE).map(normalize, num_parallel_calls=tf.data.experimental.AUTOTUNE).shuffle(len_train).batch(len_train).prefetch(len_train)
    return train_set,train_set_one_epoch,1
    for epoch in range(EPOCHS):
        train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(full_wrapper, num_parallel_calls=tf.data.experimental.AUTOTUNE).map(normalize, num_parallel_calls=tf.data.experimental.AUTOTUNE).shuffle(len_train).batch(len_train).prefetch(len_train)
#         i = 0
#         for x,y in train_set:
#             xt = x.numpy()
#             logger.debug(msg("X Shape = ",xt.shape[0], "Train Length =",len_train))
#             assert x.numpy().shape[0]==len_train
#             train_set = (xt,y.numpy())
#             i = i+1
#         assert i==1
        train_data[epoch] = train_set
        logger.debug(msg("Augmentation Epoch = ",epoch,"Time Spent = %.1f" % (time.time() - t)))
        
    
    train_set = tf.data.Dataset.from_tensor_slices(train_data[0])
    for epoch in range(1,EPOCHS):
        key = (epoch)%(max(train_data.keys())+1)
        ts = tf.data.Dataset.from_tensor_slices(train_data[key])
        train_set = train_set.concatenate(ts)
    train_set.batch(BATCH_SIZE).prefetch(len_train).shuffle(len_train).cache()
    time_taken = time.time() - t
    logger.info(msg("Augmentation Done for EPOCHS = %s, Time Taken = %.1f"%(EPOCHS,time_taken)))
    return train_set,train_set_one_epoch,time_taken
    
    

def fetch_cifar_data():
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
    classes = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']
    len_train, len_test = len(x_train), len(x_test)
    y_train = y_train.astype('int64').reshape(len_train)
    y_test = y_test.astype('int64').reshape(len_test)
    train_mean = np.mean(x_train, axis=(0,1,2))
    train_std = np.std(x_train, axis=(0,1,2))

    def normalize(x):
        return ((x.astype('float32') - train_mean) / train_std).astype('float32')

    
    pad4 = lambda x: np.pad(x, [(0, 0), (4, 4), (4, 4), (0, 0)], mode='reflect')
    x_train = pad4(x_train)
    x_test = normalize(x_test)
    logger.info("Data Fetching Done")
    return x_train, y_train, x_test, y_test, classes,len_train,len_test, normalize, train_mean


     





def runner(configs,cv=3):
    # fetch Data
    x_train, y_train, x_test, y_test, classes,len_train,len_test, normalize,train_mean = fetch_cifar_data()
    prev_da_config = None
    train_data = None
    results = []
    logger.info("Started GridSearch, Total Configs to Test = %s"%len(configs))
    reset_keras()
    t = time.time()
    prev_epochs = 0
    for i,config in enumerate(configs):
        
        _ = gc.collect()
        model_config = config["model_config"]
        da_config = config["augmentation_config"]
        BATCH_SIZE = config["training_config"]["BATCH_SIZE"]
        EPOCHS = config["training_config"]["EPOCHS"]
        save_file = config["training_config"]["save_file"]
        
        
        same_as_previous_config = prev_da_config == da_config and EPOCHS==prev_epochs
        prev_da_config = da_config
        prev_epochs = EPOCHS
        
        if not same_as_previous_config:
            augmenter = build_augmenters(da_config,train_mean)
            train_set,train_set_one_epoch,time_taken = process_full_augmentation_all_epochs(augmenter, EPOCHS,BATCH_SIZE, normalize, x_train, y_train)
            
        train_acc = test_acc = time_spent = 0.0
        for j in range(cv):
            model,opt,global_step = model_builder(model_config,BATCH_SIZE, len_train)
            t1_acc,t2_acc,tsp = run_model(model,BATCH_SIZE,EPOCHS,train_set,train_set_one_epoch,x_test, y_test,len_train,log_test_acc=False)
            train_acc += t1_acc
            test_acc += t2_acc
            time_spent += tsp
            reset_keras()
        train_acc /= cv
        test_acc /= cv
        time_spent /= cv
        
        logger.info(msg("Epoch =",i,"Train acc = ",train_acc,"Test acc =",test_acc,"Time Taken = ",time_spent))
            
        
        result = dict(config=config,results=dict(train=train_acc,test=test_acc,training_time=time_spent,augmentation_time=time_taken))
        read_and_append_to_results(save_file, result)
        results.append(result)
        del model
        model = None
    logger.info("Grid Search Complete, Total Results Count = %s, Time Taken = %.1f",len(results),(time.time()-t))
    assert len(results)==len(configs)
    return results
    



In [16]:
len(read_and_append_to_results("results.txt", read=True))

105

In [17]:
augmentation_config = dict(cutout_config=dict(s_l=0.04, s_h=0.06,max_erasures_per_image=1),
                           hue_config=dict(max_delta=0.05),
                           cifar10_augs_config=dict(proba=0.5, enabled_policies=[("rotate",5, 15,),("shearX",0.05, 0.15,),("shearY",0.05, 0.15,)]))

model_config = dict(model=dict(start_kernels=64,sparse_bn=True,thin_block=False,
                               enable_skip=True,enable_pool_before_skip=True),
                    optimizer=dict(enable_olr=True,max_lr=0.6,
                                   start_lr=0.01,pre_end_lr=0.04,end_lr=0.02,
                                   mid_epoch=5,pre_end_epoch=13,
                                   max_momentum=0.9,min_momentum=0.8, enable_momentum=True,))

training_config = dict(BATCH_SIZE=512,EPOCHS=15,save_file="results.txt")

test_config_0 = dict(augmentation_config=augmentation_config,model_config=model_config,training_config=training_config)

test_config_1 = copy.deepcopy(test_config_0)
test_config_1["model_config"]["optimizer"]["mid_epoch"] = 1
test_config_1["model_config"]["optimizer"]["pre_end_epoch"] = 2
test_config_1["training_config"]["EPOCHS"] = 3


# mid_epoch

# test_config_1 = copy.deepcopy(test_config_0)
# test_config_0["augmentation_config"]["cutout_config"]["proba"] = 0.9
# test_config_0["augmentation_config"]["cutout_config"]["s_h"] = 0.05

# 
# test_config_2 = copy.deepcopy(test_config_0)
# test_config_2["model_config"]["optimizer"]["max_momentum"] = 0.9
# test_config_2["model_config"]["optimizer"]["min_momentum"] = 0.7

# test_config_3 = copy.deepcopy(test_config_0)
# test_config_3["model_config"]["optimizer"]["pre_end_epoch"] = 14
# test_config_3["model_config"]["optimizer"]["max_momentum"] = 0.9
# test_config_3["model_config"]["optimizer"]["min_momentum"] = 0.7








results = runner([test_config_1],cv=1)


# Next Test Confirm
# Low LR - Low Momentum
# High LR - High Momentum

I0824 13:36:39.452653 140608712148800 <ipython-input-15-a595619e1ae4>:67] Data Fetching Done
I0824 13:36:39.453457 140608712148800 <ipython-input-15-a595619e1ae4>:83] Started GridSearch, Total Configs to Test = 1
W0824 13:36:45.266106 140608712148800 training_utils.py:1300] Expected a shuffled dataset but input dataset `x` is not shuffled. Please invoke `shuffle()` on input dataset.


TypeError: in converted code:


    TypeError: tf__call() missing 1 required positional argument: 'y'
